# **Librerías**

In [ ]:
# Cargar librerías
import os, mlflow
from dotenv import load_dotenv
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature
from mlflow import MlflowClient
import mlflow

# **Iniciar el experimento**

In [ ]:
load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/pipochatgpt@gmail.com/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

# **Pre-Procesamiento**

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-02.parquet')

In [4]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

In [5]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# **Gradient Boost**

In [11]:
from sklearn.ensemble import GradientBoostingRegressor

def objective_gb(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "random_state": 42
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "gradient_boosting")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento 
        model = GradientBoostingRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val[:5].toarray(), y_pred[:5])

         # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.sklearn.log_model(
            model,
            name="model",
            input_example=pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out()),
            signature=signature
        )      

    # Optuna minimiza el valor retornado
    return rmse

In [ ]:
mlflow.sklearn.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study_gb = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------

with mlflow.start_run(run_name="Gradient Boosting Hyperparameter Optimization (Optuna)", nested=True):
    study_gb.optimize(objective_gb, n_trials=3)

        # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study_gb.best_params
    best_params["random_state"] = 42

    mlflow.log_params(best_params)
    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "gradient_boosting",
        "feature_set_version": 1,
    })

     # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    
    model = GradientBoostingRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out())
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.sklearn.log_model(model, name="model", input_example=input_example, signature=signature)

[I 2025-10-23 20:50:11,777] A new study created in memory with name: no-name-b8b3a0ce-de10-4a93-b575-88bc3c6c768d
2025/10/23 20:50:36 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/10/23 20:50:39 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-23 20:50:44,079] Trial 0 finished with value: 5.423033869318215 and parameters: {'n_estimators': 69, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 0 with value: 5.423033869318215.


🏃 View run sassy-bat-164 at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394/runs/b530df804ced4a0bafee30c68447c79f
🧪 View experiment at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394


2025/10/23 20:51:07 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/10/23 20:51:09 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-23 20:51:15,806] Trial 1 finished with value: 6.667930901959873 and parameters: {'n_estimators': 57, 'learning_rate': 0.012184186502221764, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 8}. Best is trial 0 with value: 5.423033869318215.


...

2025/10/23 20:51:35 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-23 20:51:39,569] Trial 2 finished with value: 5.418235873807679 and parameters: {'n_estimators': 51, 'learning_rate': 0.2708160864249968, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 2 with value: 5.418235873807679.


🏃 View run vaunted-donkey-218 at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394/runs/49f89b5478004e34a1fa5cfd131fe3a2
🧪 View experiment at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394


2025/10/23 20:52:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\diana\Downloads\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
2025/10/23 20:52:04 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run Gradient Boosting Hyperparameter Optimization (Optuna) at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394/runs/ce0a5bdf9c1741838e700c47623bc577
🧪 View experiment at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394


# **Random Forest**

In [25]:
from sklearn.ensemble import RandomForestRegressor

mlflow.sklearn.autolog(log_models=False)

def objective_rf(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 100),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "random_state": 42,
        "n_jobs": -1
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "random_forest")
        mlflow.log_params(params)

        # Entrenamiento 
        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val[:5].toarray(), y_pred[:5])
        
        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.sklearn.log_model(
            model,
            name="model",
            input_example=pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out()),
            signature=signature
        )

    return rmse

In [26]:
# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study_rf = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------

with mlflow.start_run(run_name="Random Forest Hyperparameter Optimization (Optuna)", nested=True):
    study_rf.optimize(objective_rf, n_trials=3)

    # Recuperar y registrar los mejores hiperparámetros
    best_params = study_rf.best_params
    best_params["random_state"] = 42

    mlflow.log_params(best_params)
    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "random_forest",
        "feature_set_version": 1,
    })

     # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    
    model = RandomForestRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

     # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out())
    signature = infer_signature(input_example, y_val[:5])

    mlflow.sklearn.log_model(model, name="model", input_example=input_example, signature=signature)

[I 2025-10-23 21:22:46,916] A new study created in memory with name: no-name-3a0f6175-d319-4e39-855d-1ad09b455b75
2025/10/23 21:23:15 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/10/23 21:23:19 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-23 21:23:40,068] Trial 0 finished with value: 5.535684811605542 and parameters: {'n_estimators': 69, 'max_depth': 29, 'min_samples_split': 8, 'min_samples_leaf': 6}. Best is trial 0 with value: 5.535684811605542.


🏃 View run lyrical-shark-666 at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394/runs/d21a2e2c4e6d47dab035cf2557479509
🧪 View experiment at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394


2025/10/23 21:23:57 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/10/23 21:24:00 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-23 21:24:06,106] Trial 1 finished with value: 5.591476239254621 and parameters: {'n_estimators': 57, 'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 9}. Best is trial 0 with value: 5.535684811605542.


...

2025/10/23 21:24:31 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run fearless-bee-844 at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394/runs/1223c4ffa70b48919920d9f11a795c93
🧪 View experiment at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394


[I 2025-10-23 21:24:48,014] Trial 2 finished with value: 5.600874046133536 and parameters: {'n_estimators': 80, 'max_depth': 23, 'min_samples_split': 2, 'min_samples_leaf': 10}. Best is trial 0 with value: 5.535684811605542.
2025/10/23 21:25:38 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\diana\Downloads\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
2025/10/23 21:25:43 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run Random Forest Hyperparameter Optimization (Optuna) at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394/runs/3df89e10d31541e096c1c069122f918a
🧪 View experiment at: https://dbc-c600c0c2-acad.cloud.databricks.com/ml/experiments/1588665197670394


# **Registrar modelo en Model Registry**
El mejor de los dos modelos fue el de Gradient Boosting, por lo que usaremos ese como challenger:

In [22]:
model_name = "workspace.default.nyc-taxi-model"

In [21]:
client = MlflowClient()

MODEL_FAMILY = "gradient_boosting"

# Buscar todos los runs del experimento
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    filter_string=f"tags.model_family = '{MODEL_FAMILY}' and attribute.status = 'FINISHED'",
    order_by=["metrics.rmse ASC"]
)

# Filtrar solo los runs padres (sin parentRunId)
padre = runs[runs["tags.mlflow.parentRunId"].isnull()]

# Mostrar el mejor padre
if len(padre) > 0:
    best_run = padre.iloc[0]
    print(f"🏆 Mejor run padre para {MODEL_FAMILY}:")
    print(f"Run Name: {best_run['tags.mlflow.runName']}")
    print(f"Run ID: {best_run['run_id']}")
    print(f"RMSE: {best_run['metrics.rmse']:.4f}")
else:
    print(f"No se encontraron runs padres para {MODEL_FAMILY}.")


🏆 Mejor run padre para gradient_boosting:
Run Name: Gradient Boosting Hyperparameter Optimization (Optuna)
Run ID: ce0a5bdf9c1741838e700c47623bc577
RMSE: 5.4182


In [24]:
# Registrar el mejor modelo
result = mlflow.register_model(
    model_uri=f"runs:/{best_run['run_id']}/model",
    name=model_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/23 21:08:32 WARNING mlflow.tracking._model_registry.fluent: Run with id ce0a5bdf9c1741838e700c47623bc577 has no artifacts at artifact path 'model', registering model based on models:/m-27df69109ebe4beea0d91f15a3f18b89 instead


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Created version '2' of model 'workspace.default.nyc-taxi-model'.


In [27]:
model_version = result.version
new_alias = "Challenger"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

# **Comparar Modelos**

In [28]:
import mlflow.pyfunc
champion_uri = f"models:/{model_name}@Champion"
challenger_uri = f"models:/{model_name}@Challenger"

champion_version = mlflow.pyfunc.load_model(champion_uri)
challenger_version = mlflow.pyfunc.load_model(challenger_uri)

c:\Users\diana\Downloads\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:31:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


In [41]:
df_val = read_dataframe('../data/green_tripdata_2025-03.parquet')
X_val = preprocess(df_val, dv)
target = 'duration'
y_val = df_val[target].values

In [43]:
challenger_version.predict(X_train)

MlflowException: Failed to enforce schema of data '<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 92614 stored elements and shape (46307, 4159)>
  Coords	Values
  (0, 3387)	1.0
  (0, 4158)	5.93
  (1, 824)	1.0
  (1, 4158)	1.32
  (2, 888)	1.0
  (2, 4158)	0.41
  (3, 3245)	1.0
  (3, 4158)	4.12
  (4, 3030)	1.0
  (4, 4158)	4.71
  (5, 2150)	1.0
  (5, 4158)	4.26
  (6, 3544)	1.0
  (6, 4158)	2.0
  (7, 803)	1.0
  (7, 4158)	2.09
  (8, 762)	1.0
  (8, 4158)	0.95
  (9, 3388)	1.0
  (9, 4158)	0.66
  (10, 2540)	1.0
  (10, 4158)	0.0
  (11, 2452)	1.0
  (11, 4158)	1.7
  (12, 3412)	1.0
  :	:
  (46294, 4158)	1.91
  (46295, 4068)	1.0
  (46295, 4158)	4.24
  (46296, 230)	1.0
  (46296, 4158)	1.21
  (46297, 3328)	1.0
  (46297, 4158)	8.1
  (46298, 1170)	1.0
  (46298, 4158)	0.74
  (46299, 1170)	1.0
  (46299, 4158)	0.84
  (46300, 3101)	1.0
  (46300, 4158)	1.52
  (46301, 2463)	1.0
  (46301, 4158)	7.11
  (46302, 924)	1.0
  (46302, 4158)	13.99
  (46303, 824)	1.0
  (46303, 4158)	1.51
  (46304, 2497)	1.0
 

In [40]:
df_new = read_dataframe('../data/green_tripdata_2025-03.parquet')

# Preprocesar con el DictVectorizer que guardaste
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

X_new = preprocess(df_new, dv)  # Esto devuelve matriz sparse

X_new_df = pd.DataFrame(X_new.toarray(), columns=dv.get_feature_names_out())
y_pred = champion_version.predict(X_new_df)

MlflowException: Failed to enforce schema of data '       PU_DO=101_258  PU_DO=101_82  PU_DO=102_112  PU_DO=102_130  \
0                0.0           0.0            0.0            0.0   
1                0.0           0.0            0.0            0.0   
2                0.0           0.0            0.0            0.0   
3                0.0           0.0            0.0            0.0   
4                0.0           0.0            0.0            0.0   
...              ...           ...            ...            ...   
48331            0.0           0.0            0.0            0.0   
48332            0.0           0.0            0.0            0.0   
48333            0.0           0.0            0.0            0.0   
48334            0.0           0.0            0.0            0.0   
48335            0.0           0.0            0.0            0.0   

       PU_DO=102_236  PU_DO=102_28  PU_DO=102_82  PU_DO=102_95  PU_DO=106_123  \
0                0.0           0.0           0.0   

In [29]:
df_val = read_dataframe('../data/green_tripdata_2025-03.parquet')
X_val = preprocess(df_val, dv)
target = 'duration'
y_val = df_val[target].values

In [38]:
val_dicts = df_val[['PULocationID', 'DOLocationID', 'trip_distance']].astype(str)
preds = champion_version.predict(val_dicts)

MlflowException: Failed to enforce schema of data '      PULocationID DOLocationID trip_distance
0               75          239           2.2
1               41           42          1.06
2              265           56         18.91
3               82          236          8.36
4               66           33          0.82
...            ...          ...           ...
51534           25          106          1.57
51535           74          262          2.31
51536           65           79          3.57
51537           75          265         13.51
51538          244           74          4.64

[48336 rows x 3 columns]' with schema '['DOLocationID=1': double (required), 'DOLocationID=10': double (required), 'DOLocationID=100': double (required), 'DOLocationID=101': double (required), 'DOLocationID=102': double (required), 'DOLocationID=106': double (required), 'DOLocationID=107': double (required), 'DOLocationID=108': double (required), 'DOLocationID=11': double (required), 'DOLocati